In [1]:
%matplotlib inline



Relational graph convolutional network
================================================

**Author:** Lingfan Yu, Mufei Li, Zheng Zhang

In this tutorial, you learn how to implement a relational graph convolutional
network (R-GCN). This type of network is one effort to generalize GCN 
to handle different relationships between entities in a knowledge base. To 
learn more about the research behind R-GCN, see `Modeling Relational Data with Graph Convolutional
Networks <https://arxiv.org/pdf/1703.06103.pdf>`_ 

The straightforward graph convolutional network (GCN) and 
`DGL tutorial <http://doc.dgl.ai/tutorials/index.html>`_) exploits
structural information of a dataset (that is, the graph connectivity) in order to
improve the extraction of node representations. Graph edges are left as
untyped.

A knowledge graph is made up of a collection of triples in the form
subject, relation, object. Edges thus encode important information and
have their own embeddings to be learned. Furthermore, there may exist
multiple edges among any given pair.


A brief introduction to R-GCN
---------------------------
In *statistical relational learning* (SRL), there are two fundamental
tasks:

- **Entity classification** - Where you assign types and categorical
  properties to entities.
- **Link prediction** - Where you recover missing triples.

In both cases, missing information is expected to be recovered from the 
neighborhood structure of the graph. For example, the R-GCN
paper cited earlier provides the following example. Knowing that Mikhail Baryshnikov was educated at the Vaganova Academy
implies both that Mikhail Baryshnikov should have the label person, and
that the triple (Mikhail Baryshnikov, lived in, Russia) must belong to the
knowledge graph.

R-GCN solves these two problems using a common graph convolutional network. It's 
extended with multi-edge encoding to compute embedding of the entities, but
with different downstream processing.

- Entity classification is done by attaching a softmax classifier at the
  final embedding of an entity (node). Training is through loss of standard
  cross-entropy.
- Link prediction is done by reconstructing an edge with an autoencoder
  architecture, using a parameterized score function. Training uses negative
  sampling.

This tutorial focuses on the first task, entity classification, to show how to generate entity
representation. `Complete
code <https://github.com/dmlc/dgl/tree/rgcn/examples/pytorch/rgcn>`_
for both tasks is found in the DGL Github repository.

Key ideas of R-GCN
-------------------
Recall that in GCN, the hidden representation for each node $i$ at
$(l+1)^{th}$ layer is computed by:

\begin{align}h_i^{l+1} = \sigma\left(\sum_{j\in N_i}\frac{1}{c_i} W^{(l)} h_j^{(l)}\right)~~~~~~~~~~(1)\\\end{align}

where $c_i$ is a normalization constant.

The key difference between R-GCN and GCN is that in R-GCN, edges can
represent different relations. In GCN, weight $W^{(l)}$ in equation
$(1)$ is shared by all edges in layer $l$. In contrast, in
R-GCN, different edge types use different weights and only edges of the
same relation type $r$ are associated with the same projection weight
$W_r^{(l)}$.

So the hidden representation of entities in $(l+1)^{th}$ layer in
R-GCN can be formulated as the following equation:

\begin{align}h_i^{l+1} = \sigma\left(W_0^{(l)}h_i^{(l)}+\sum_{r\in R}\sum_{j\in N_i^r}\frac{1}{c_{i,r}}W_r^{(l)}h_j^{(l)}\right)~~~~~~~~~~(2)\\\end{align}

where $N_i^r$ denotes the set of neighbor indices of node $i$
under relation $r\in R$ and $c_{i,r}$ is a normalization
constant. In entity classification, the R-GCN paper uses
$c_{i,r}=|N_i^r|$.

The problem of applying the above equation directly is the rapid growth of
the number of parameters, especially with highly multi-relational data. In
order to reduce model parameter size and prevent overfitting, the original
paper proposes to use basis decomposition.

\begin{align}W_r^{(l)}=\sum\limits_{b=1}^B a_{rb}^{(l)}V_b^{(l)}~~~~~~~~~~(3)\\\end{align}

Therefore, the weight $W_r^{(l)}$ is a linear combination of basis
transformation $V_b^{(l)}$ with coefficients $a_{rb}^{(l)}$.
The number of bases $B$ is much smaller than the number of relations
in the knowledge base.

<div class="alert alert-info"><h4>Note</h4><p>Another weight regularization, block-decomposition, is implemented in
   the `link prediction <link-prediction_>`_.</p></div>

Implement R-GCN in DGL
----------------------

An R-GCN model is composed of several R-GCN layers. The first R-GCN layer
also serves as input layer and takes in features (for example, description texts)
that are associated with node entity and project to hidden space. In this tutorial,
we only use the entity ID as an entity feature.

R-GCN layers
~~~~~~~~~~~~

For each node, an R-GCN layer performs the following steps:

- Compute outgoing message using node representation and weight matrix
  associated with the edge type (message function)
- Aggregate incoming messages and generate new node representations (reduce
  and apply function)

The following code is the definition of an R-GCN hidden layer.

<div class="alert alert-info"><h4>Note</h4><p>Each relation type is associated with a different weight. Therefore,
   the full weight matrix has three dimensions: relation, input_feature,
   output_feature.</p></div>




In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import dgl.function as fn
from functools import partial
import dgl

class RGCNLayer(nn.Module):
    def __init__(self, in_feat, out_feat, num_rels, num_bases=-1, bias=None,
                 activation=None, is_input_layer=False):
        super(RGCNLayer, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.bias = bias
        self.activation = activation
        self.is_input_layer = is_input_layer

        # sanity check
        if self.num_bases <= 0 or self.num_bases > self.num_rels:
            self.num_bases = self.num_rels

        # weight bases in equation (3)
        self.weight = nn.Parameter(torch.Tensor(self.num_bases, self.in_feat,
                                                self.out_feat))
        if self.num_bases < self.num_rels:
            # linear combination coefficients in equation (3)
            self.w_comp = nn.Parameter(torch.Tensor(self.num_rels, self.num_bases))

        # add bias
        if self.bias:
            self.bias = nn.Parameter(torch.Tensor(out_feat))

        # init trainable parameters
        nn.init.xavier_uniform_(self.weight,
                                gain=nn.init.calculate_gain('relu'))
        if self.num_bases < self.num_rels:
            nn.init.xavier_uniform_(self.w_comp,
                                    gain=nn.init.calculate_gain('relu'))
        if self.bias:
            nn.init.xavier_uniform_(self.bias,
                                    gain=nn.init.calculate_gain('relu'))

    def forward(self, g):
        if self.num_bases < self.num_rels:
            # generate all weights from bases (equation (3))
            weight = self.weight.view(self.in_feat, self.num_bases, self.out_feat)
            weight = torch.matmul(self.w_comp, weight).view(self.num_rels,
                                                        self.in_feat, self.out_feat)
        else:
            weight = self.weight

        if self.is_input_layer:
            def message_func(edges):
                # for input layer, matrix multiply can be converted to be
                # an embedding lookup using source node id
                embed = weight.view(-1, self.out_feat)
                index = edges.data['rel_type'] * self.in_feat + edges.src['id']
                return {'msg': embed[index] * edges.data['norm']}
        else:
            def message_func(edges):
                w = weight[edges.data['rel_type'].long()]
                msg = torch.bmm(edges.src['h'].unsqueeze(1), w).squeeze()
                msg = msg * edges.data['norm']
                return {'msg': msg}

        def apply_func(nodes):
            h = nodes.data['h']
            if self.bias:
                h = h + self.bias
            if self.activation:
                h = self.activation(h)
            return {'h': h}

        g.update_all(message_func, fn.sum(msg='msg', out='h'), apply_func)

Using backend: pytorch


Full R-GCN model defined
~~~~~~~~~~~~~~~~~~~~~~~



In [3]:
class Model(nn.Module):
    def __init__(self, num_nodes, h_dim, out_dim, num_rels,
                 num_bases=-1, num_hidden_layers=1):
        super(Model, self).__init__()
        self.num_nodes = num_nodes
        self.h_dim = h_dim
        self.out_dim = out_dim
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.num_hidden_layers = num_hidden_layers

        # create rgcn layers
        self.build_model()

        # create initial features
        self.features = self.create_features()

    def build_model(self):
        self.layers = nn.ModuleList()
        # input to hidden
        i2h = self.build_input_layer()
        self.layers.append(i2h)
        # hidden to hidden
        for _ in range(self.num_hidden_layers):
            h2h = self.build_hidden_layer()
            self.layers.append(h2h)
        # hidden to output
        h2o = self.build_output_layer()
        self.layers.append(h2o)

    # initialize feature for each node
    def create_features(self):
        features = torch.arange(self.num_nodes)
        return features

    def build_input_layer(self):
        return RGCNLayer(self.num_nodes, self.h_dim, self.num_rels, self.num_bases,
                         activation=F.relu, is_input_layer=True)

    def build_hidden_layer(self):
        return RGCNLayer(self.h_dim, self.h_dim, self.num_rels, self.num_bases,
                         activation=F.relu)

    def build_output_layer(self):
        return RGCNLayer(self.h_dim, self.out_dim, self.num_rels, self.num_bases,
                         activation=partial(F.softmax, dim=1))

    def forward(self, g):
        if self.features is not None:
            g.ndata['id'] = self.features
        for layer in self.layers:
            layer(g)
        return g.ndata.pop('h')

Handle dataset
~~~~~~~~~~~~~~~~
This tutorial uses Institute for Applied Informatics and Formal Description Methods (AIFB) dataset from R-GCN paper.



In [4]:
# load graph data
from dgl.contrib.data import load_data
import numpy as np
data = load_data(dataset='aifb')
num_nodes = data.num_nodes
num_rels = data.num_rels
num_classes = data.num_classes
labels = data.labels
train_idx = data.train_idx
# split training and validation set
val_idx = train_idx[:len(train_idx) // 5]
train_idx = train_idx[len(train_idx) // 5:]

# edge type and normalization factor
edge_type = torch.from_numpy(data.edge_type)
edge_norm = torch.from_numpy(data.edge_norm).unsqueeze(1)

labels = torch.from_numpy(labels).view(-1)

Loading dataset aifb
Number of nodes:  8285
Number of edges:  66371
Number of relations:  91
Number of classes:  4
removing nodes that are more than 3 hops away


Create graph and model
~~~~~~~~~~~~~~~~~~~~~~~



In [5]:
len(data.labels)

8285

In [6]:
for i, j in zip(data.edge_src, data.edge_dst):
    print(i, j)

0 0
919 0
919 0
1127 0
1127 0
1934 0
2062 0
5286 0
5286 0
7786 0
7786 0
1 1
1534 1
2 2
3577 2
3 3
7656 3
4 4
377 4
1028 4
1148 4
1148 4
1174 4
1384 4
1384 4
3298 4
3722 4
3822 4
3822 4
5080 4
5080 4
5176 4
5176 4
5354 4
5667 4
5961 4
7056 4
5 5
49 5
6 6
7519 6
7 7
7708 7
8 8
377 8
592 8
1668 8
2364 8
3705 8
4362 8
4583 8
5761 8
5761 8
6110 8
7056 8
9 9
2724 9
10 10
6516 10
11 11
3049 11
12 12
1934 12
3060 12
3060 12
5474 12
5474 12
7229 12
7229 12
7836 12
8274 12
8274 12
13 13
4757 13
14 14
5979 14
15 15
7005 15
16 16
1357 16
17 17
1490 17
1934 17
2682 17
3765 17
3765 17
18 18
63 18
63 18
1934 18
7693 18
19 19
7242 19
20 20
2966 20
21 21
7884 21
22 22
5355 22
23 23
2374 23
24 24
2369 24
25 25
7255 25
26 26
2453 26
27 27
60 27
28 28
1606 28
2104 28
29 29
6855 29
30 30
3193 30
31 31
2661 31
32 32
4790 32
33 33
277 33
944 33
962 33
1174 33
2028 33
2028 33
2672 33
3823 33
4123 33
6024 33
6024 33
6260 33
6260 33
6631 33
6631 33
7008 33
7056 33
7264 33
34 34
7980 34
35 35
8192 35
36 36
2359 

2629 169
2672 169
2672 169
3009 169
3009 169
3150 169
3150 169
3181 169
3181 169
3531 169
3531 169
3554 169
3554 169
4462 169
4462 169
6024 169
6024 169
6207 169
6260 169
6260 169
7264 169
7264 169
7315 169
7315 169
7373 169
7373 169
7536 169
7815 169
7815 169
170 170
6478 170
171 171
5302 171
172 172
2090 172
173 173
5644 173
174 174
218 174
1911 174
1945 174
2689 174
2689 174
3298 174
4344 174
4933 174
4933 174
6142 174
6677 174
7056 174
7703 174
7703 174
8080 174
175 175
3765 175
176 176
441 176
1010 176
2130 176
4463 176
5612 176
5649 176
5796 176
6781 176
7578 176
7699 176
177 177
1934 177
4826 177
4826 177
4920 177
4920 177
5558 177
178 178
7643 178
179 179
377 179
1174 179
1655 179
2074 179
3197 179
3512 179
3998 179
3998 179
4250 179
4250 179
4274 179
5080 179
5080 179
6049 179
6633 179
7056 179
180 180
1934 180
6239 180
6239 180
6418 180
182 182
377 182
529 182
1395 182
1399 182
3298 182
3998 182
3998 182
4362 182
5569 182
5667 182
5726 182
6164 182
6164 182
7056 182
183 183
1

541 302
1131 302
1131 302
1380 302
2206 302
2712 302
2712 302
3324 302
3624 302
3624 302
3626 302
3677 302
3677 302
3916 302
3916 302
3942 302
4032 302
4032 302
4881 302
4881 302
5667 302
6142 302
6346 302
6346 302
6909 302
6909 302
7056 302
7934 302
8194 302
303 303
377 303
1174 303
1344 303
2165 303
2165 303
2235 303
3688 303
3722 303
7056 303
304 304
1686 304
1686 304
1934 304
3660 304
3660 304
7106 304
305 305
3845 305
4463 305
4959 305
5612 305
306 306
838 306
4748 306
5622 306
6207 306
307 307
4296 307
308 308
8234 308
309 309
6496 309
310 310
6518 310
311 311
4099 311
277 312
312 312
377 312
396 312
396 312
428 312
428 312
1174 312
1622 312
2165 312
2165 312
3071 312
3298 312
4044 312
5009 312
5009 312
5047 312
5443 312
5443 312
6889 312
7056 312
7196 312
7196 312
7519 312
7519 312
8250 312
313 313
1770 313
1770 313
1934 313
6584 313
315 315
6069 315
316 316
377 316
1395 316
1446 316
1446 316
2656 316
3298 316
3998 316
3998 316
4362 316
5080 316
5080 316
6164 316
6164 316
6799 3

In [ ]:
len(data.edge_dst),len(data.edge_src)

In [ ]:
# configurations
n_hidden = 16 # number of hidden units
n_bases = -1 # use number of relations as number of bases
n_hidden_layers = 0 # use 1 input layer, 1 output layer, no hidden layer
n_epochs = 25 # epochs to train
lr = 0.01 # learning rate
l2norm = 0 # L2 norm coefficient

# create graph
g = DGLGraph((data.edge_src, data.edge_dst))
g.edata.update({'rel_type': edge_type, 'norm': edge_norm})

# create model
model = Model(len(g),
              n_hidden,
              num_classes,
              num_rels,
              num_bases=n_bases,
              num_hidden_layers=n_hidden_layers)

In [9]:
g.edges(),g.nodes()

((tensor([   0,  919,  919,  ..., 5939, 5939, 8284]),
  tensor([   0,    0,    0,  ..., 8284, 8284, 8284])),
 tensor([   0,    1,    2,  ..., 8282, 8283, 8284]))

Training loop
~~~~~~~~~~~~~~~~



In [12]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2norm)

print("start training...")
model.train()
for epoch in range(n_epochs):
    optimizer.zero_grad()
    logits = model.forward(g)
    loss = F.cross_entropy(logits[train_idx], labels[train_idx].long())
    loss.backward()

    optimizer.step()

    train_acc = torch.sum(logits[train_idx].argmax(dim=1) == labels[train_idx])
    train_acc = train_acc.item() / len(train_idx)
    val_loss = F.cross_entropy(logits[val_idx], labels[val_idx].long())
    val_acc = torch.sum(logits[val_idx].argmax(dim=1) == labels[val_idx])
    val_acc = val_acc.item() / len(val_idx)
    print("Epoch {:05d} | ".format(epoch) +
          "Train Accuracy: {:.4f} | Train Loss: {:.4f} | ".format(
              train_acc, loss.item()) +
          "Validation Accuracy: {:.4f} | Validation loss: {:.4f}".format(
              val_acc, val_loss.item()))

start training...
Epoch 00000 | Train Accuracy: 0.1429 | Train Loss: 1.3868 | Validation Accuracy: 0.1786 | Validation loss: 1.3866
Epoch 00001 | Train Accuracy: 0.9554 | Train Loss: 1.3479 | Validation Accuracy: 1.0000 | Validation loss: 1.3601
Epoch 00002 | Train Accuracy: 0.9554 | Train Loss: 1.2883 | Validation Accuracy: 1.0000 | Validation loss: 1.3200
Epoch 00003 | Train Accuracy: 0.9554 | Train Loss: 1.2108 | Validation Accuracy: 1.0000 | Validation loss: 1.2657
Epoch 00004 | Train Accuracy: 0.9464 | Train Loss: 1.1272 | Validation Accuracy: 1.0000 | Validation loss: 1.2005
Epoch 00005 | Train Accuracy: 0.9554 | Train Loss: 1.0516 | Validation Accuracy: 1.0000 | Validation loss: 1.1306
Epoch 00006 | Train Accuracy: 0.9643 | Train Loss: 0.9899 | Validation Accuracy: 1.0000 | Validation loss: 1.0632
Epoch 00007 | Train Accuracy: 0.9821 | Train Loss: 0.9412 | Validation Accuracy: 1.0000 | Validation loss: 1.0039
Epoch 00008 | Train Accuracy: 0.9821 | Train Loss: 0.9022 | Validation


The second task, link prediction
--------------------------------
So far, you have seen how to use DGL to implement entity classification with an 
R-GCN model. In the knowledge base setting, representation generated by
R-GCN can be used to uncover potential relationships between nodes. In the 
R-GCN paper, the authors feed the entity representations generated by R-GCN
into the `DistMult <https://arxiv.org/pdf/1412.6575.pdf>`_ prediction model
to predict possible relationships.

The implementation is similar to that presented here, but with an extra DistMult layer
stacked on top of the R-GCN layers. You can find the complete
implementation of link prediction with R-GCN in our `Github Python code example
 <https://github.com/dmlc/dgl/blob/master/examples/pytorch/rgcn/link_predict.py>`_.



In [45]:
from dgl.nn.pytorch import RelGraphConv
class BaseRGCN(nn.Module):
    def __init__(self, num_nodes, h_dim, out_dim, num_rels, num_bases,
                 num_hidden_layers=1, dropout=0,
                 use_self_loop=False, use_cuda=False):
        super(BaseRGCN, self).__init__()
        self.num_nodes = num_nodes
        self.h_dim = h_dim
        self.out_dim = out_dim
        self.num_rels = num_rels
        self.num_bases = None if num_bases < 0 else num_bases
        self.num_hidden_layers = num_hidden_layers
        self.dropout = dropout
        self.use_self_loop = use_self_loop
        self.use_cuda = use_cuda

        # create rgcn layers
        self.build_model()

    def build_model(self):
        self.layers = nn.ModuleList()
        # i2h
        i2h = self.build_input_layer()
        if i2h is not None:
            self.layers.append(i2h)
        # h2h
        for idx in range(self.num_hidden_layers):
            h2h = self.build_hidden_layer(idx)
            self.layers.append(h2h)
        # h2o
        h2o = self.build_output_layer()
        if h2o is not None:
            self.layers.append(h2o)

    def build_input_layer(self):
        return None

    def build_hidden_layer(self, idx):
        raise NotImplementedError

    def build_output_layer(self):
        return None

    def forward(self, g, h, r, norm):
        for layer in self.layers:
            h = layer(g, h, r, norm)
        return h

In [46]:
class EmbeddingLayer(nn.Module):
    def __init__(self, num_nodes, h_dim):
        super(EmbeddingLayer, self).__init__()
        self.embedding = torch.nn.Embedding(num_nodes, h_dim)

    def forward(self, g, h, r, norm):
        return self.embedding(h.squeeze())

class RGCN(BaseRGCN):
    def build_input_layer(self):
        return EmbeddingLayer(self.num_nodes, self.h_dim)

    def build_hidden_layer(self, idx):
        act = F.relu if idx < self.num_hidden_layers - 1 else None
        return RelGraphConv(self.h_dim, self.h_dim, self.num_rels, "bdd",
                self.num_bases, activation=act, self_loop=True,
                dropout=self.dropout)

class LinkPredict(nn.Module):
    def __init__(self, in_dim, h_dim, num_rels, num_bases=-1,
                 num_hidden_layers=1, dropout=0, use_cuda=False, reg_param=0):
        super(LinkPredict, self).__init__()
        self.rgcn = RGCN(in_dim, h_dim, h_dim, num_rels * 2, num_bases,
                         num_hidden_layers, dropout, use_cuda)
        self.reg_param = reg_param
        self.w_relation = nn.Parameter(torch.Tensor(num_rels, h_dim))
        nn.init.xavier_uniform_(self.w_relation,
                                gain=nn.init.calculate_gain('relu'))

    def calc_score(self, embedding, triplets):
        # DistMult
        s = embedding[triplets[:,0]]
        r = self.w_relation[triplets[:,1]]
        o = embedding[triplets[:,2]]
        score = torch.sum(s * r * o, dim=1)
        return score

    def forward(self, g, h, r, norm):
        return self.rgcn.forward(g, h, r, norm)

    def regularization_loss(self, embedding):
        return torch.mean(embedding.pow(2)) + torch.mean(self.w_relation.pow(2))

    def get_loss(self, g, embed, triplets, labels):
        # triplets is a list of data samples (positive and negative)
        # each row in the triplets is a 3-tuple of (source, relation, destination)
        score = self.calc_score(embed, triplets)
        predict_loss = F.binary_cross_entropy_with_logits(score, labels)
        reg_loss = self.regularization_loss(embed)
        return predict_loss + self.reg_param * reg_loss

def node_norm_to_edge_norm(g, node_norm):
    g = g.local_var()
    # convert to edge norm
    g.ndata['norm'] = node_norm
    g.apply_edges(lambda edges : {'norm' : edges.dst['norm']})
    return g.edata['norm']

In [47]:
model = LinkPredict(num_nodes,
                        500,
                        num_rels,
                        num_bases=100,
                        num_hidden_layers=2,
                        dropout=0.2,
                        use_cuda=-1,
                        reg_param=0.01)

In [48]:
data = load_data('FB15k-237')

# entities: 14541
# relations: 237
# edges: 272115


In [1]:
num_nodes = data.num_nodes
train_data = data.train
valid_data = data.valid
test_data = data.test
num_rels = data.num_rels

NameError: name 'data' is not defined

In [50]:
valid_data = torch.LongTensor(valid_data)
test_data = torch.LongTensor(test_data)

In [51]:
def get_adj_and_degrees(num_nodes, triplets):
    """ Get adjacency list and degrees of the graph
    """
    adj_list = [[] for _ in range(num_nodes)]
    for i,triplet in enumerate(triplets):
        adj_list[triplet[0]].append([i, triplet[2]])
        adj_list[triplet[2]].append([i, triplet[0]])

    degrees = np.array([len(a) for a in adj_list])
    adj_list = [np.array(a) for a in adj_list]
    return adj_list, degrees

def sample_edge_neighborhood(adj_list, degrees, n_triplets, sample_size):
    """Sample edges by neighborhool expansion.
    This guarantees that the sampled edges form a connected graph, which
    may help deeper GNNs that require information from more than one hop.
    """
    edges = np.zeros((sample_size), dtype=np.int32)

    #initialize
    sample_counts = np.array([d for d in degrees])
    picked = np.array([False for _ in range(n_triplets)])
    seen = np.array([False for _ in degrees])

    for i in range(0, sample_size):
        weights = sample_counts * seen

        if np.sum(weights) == 0:
            weights = np.ones_like(weights)
            weights[np.where(sample_counts == 0)] = 0

        probabilities = (weights) / np.sum(weights)
        chosen_vertex = np.random.choice(np.arange(degrees.shape[0]),
                                         p=probabilities)
        chosen_adj_list = adj_list[chosen_vertex]
        seen[chosen_vertex] = True

        chosen_edge = np.random.choice(np.arange(chosen_adj_list.shape[0]))
        chosen_edge = chosen_adj_list[chosen_edge]
        edge_number = chosen_edge[0]

        while picked[edge_number]:
            chosen_edge = np.random.choice(np.arange(chosen_adj_list.shape[0]))
            chosen_edge = chosen_adj_list[chosen_edge]
            edge_number = chosen_edge[0]

        edges[i] = edge_number
        other_vertex = chosen_edge[1]
        picked[edge_number] = True
        sample_counts[chosen_vertex] -= 1
        sample_counts[other_vertex] -= 1
        seen[other_vertex] = True

    return edges

def sample_edge_uniform(adj_list, degrees, n_triplets, sample_size):
    """Sample edges uniformly from all the edges."""
    all_edges = np.arange(n_triplets)
    return np.random.choice(all_edges, sample_size, replace=False)

def generate_sampled_graph_and_labels(triplets, sample_size, split_size,
                                      num_rels, adj_list, degrees,
                                      negative_rate, sampler="uniform"):
    """Get training graph and signals
    First perform edge neighborhood sampling on graph, then perform negative
    sampling to generate negative samples
    """
    # perform edge neighbor sampling
    if sampler == "uniform":
        edges = sample_edge_uniform(adj_list, degrees, len(triplets), sample_size)
    elif sampler == "neighbor":
        edges = sample_edge_neighborhood(adj_list, degrees, len(triplets), sample_size)
    else:
        raise ValueError("Sampler type must be either 'uniform' or 'neighbor'.")

    # relabel nodes to have consecutive node ids
    edges = triplets[edges]
    src, rel, dst = edges.transpose()
    uniq_v, edges = np.unique((src, dst), return_inverse=True)
    src, dst = np.reshape(edges, (2, -1))
    relabeled_edges = np.stack((src, rel, dst)).transpose()

    # negative sampling
    samples, labels = negative_sampling(relabeled_edges, len(uniq_v),
                                        negative_rate)

    # further split graph, only half of the edges will be used as graph
    # structure, while the rest half is used as unseen positive samples
    split_size = int(sample_size * split_size)
    graph_split_ids = np.random.choice(np.arange(sample_size),
                                       size=split_size, replace=False)
    src = src[graph_split_ids]
    dst = dst[graph_split_ids]
    rel = rel[graph_split_ids]

    # build DGL graph
    print("# sampled nodes: {}".format(len(uniq_v)))
    print("# sampled edges: {}".format(len(src) * 2))
    g, rel, norm = build_graph_from_triplets(len(uniq_v), num_rels,
                                             (src, rel, dst))
    return g, uniq_v, rel, norm, samples, labels

def comp_deg_norm(g):
    g = g.local_var()
    in_deg = g.in_degrees(range(g.number_of_nodes())).float().numpy()
    norm = 1.0 / in_deg
    norm[np.isinf(norm)] = 0
    return norm

def build_graph_from_triplets(num_nodes, num_rels, triplets):
    """ Create a DGL graph. The graph is bidirectional because RGCN authors
        use reversed relations.
        This function also generates edge type and normalization factor
        (reciprocal of node incoming degree)
    """
    g = dgl.DGLGraph()
    g.add_nodes(num_nodes)
    src, rel, dst = triplets
    src, dst = np.concatenate((src, dst)), np.concatenate((dst, src))
    rel = np.concatenate((rel, rel + num_rels))
    edges = sorted(zip(dst, src, rel))
    dst, src, rel = np.array(edges).transpose()
    g.add_edges(src, dst)
    norm = comp_deg_norm(g)
    print("# nodes: {}, # edges: {}".format(num_nodes, len(src)))
    return g, rel.astype('int64'), norm.astype('int64')

def build_test_graph(num_nodes, num_rels, edges):
    src, rel, dst = edges.transpose()
    print("Test graph:")
    return build_graph_from_triplets(num_nodes, num_rels, (src, rel, dst))

def negative_sampling(pos_samples, num_entity, negative_rate):
    size_of_batch = len(pos_samples)
    num_to_generate = size_of_batch * negative_rate
    neg_samples = np.tile(pos_samples, (negative_rate, 1))
    labels = np.zeros(size_of_batch * (negative_rate + 1), dtype=np.float32)
    labels[: size_of_batch] = 1
    values = np.random.randint(num_entity, size=num_to_generate)
    choices = np.random.uniform(size=num_to_generate)
    subj = choices > 0.5
    obj = choices <= 0.5
    neg_samples[subj, 0] = values[subj]
    neg_samples[obj, 2] = values[obj]

    return np.concatenate((pos_samples, neg_samples)), labels

#######################################################################
#
# Utility functions for evaluations (raw)
#
#######################################################################

def sort_and_rank(score, target):
    _, indices = torch.sort(score, dim=1, descending=True)
    indices = torch.nonzero(indices == target.view(-1, 1))
    indices = indices[:, 1].view(-1)
    return indices

def perturb_and_get_raw_rank(embedding, w, a, r, b, test_size, batch_size=100):
    """ Perturb one element in the triplets
    """
    n_batch = (test_size + batch_size - 1) // batch_size
    ranks = []
    for idx in range(n_batch):
        print("batch {} / {}".format(idx, n_batch))
        batch_start = idx * batch_size
        batch_end = min(test_size, (idx + 1) * batch_size)
        batch_a = a[batch_start: batch_end]
        batch_r = r[batch_start: batch_end]
        emb_ar = embedding[batch_a] * w[batch_r]
        emb_ar = emb_ar.transpose(0, 1).unsqueeze(2) # size: D x E x 1
        emb_c = embedding.transpose(0, 1).unsqueeze(1) # size: D x 1 x V
        # out-prod and reduce sum
        out_prod = torch.bmm(emb_ar, emb_c) # size D x E x V
        score = torch.sum(out_prod, dim=0) # size E x V
        score = torch.sigmoid(score)
        target = b[batch_start: batch_end]
        ranks.append(sort_and_rank(score, target))
    return torch.cat(ranks)

# return MRR (raw), and Hits @ (1, 3, 10)
def calc_raw_mrr(embedding, w, test_triplets, hits=[], eval_bz=100):
    with torch.no_grad():
        s = test_triplets[:, 0]
        r = test_triplets[:, 1]
        o = test_triplets[:, 2]
        test_size = test_triplets.shape[0]

        # perturb subject
        ranks_s = perturb_and_get_raw_rank(embedding, w, o, r, s, test_size, eval_bz)
        # perturb object
        ranks_o = perturb_and_get_raw_rank(embedding, w, s, r, o, test_size, eval_bz)

        ranks = torch.cat([ranks_s, ranks_o])
        ranks += 1 # change to 1-indexed

        mrr = torch.mean(1.0 / ranks.float())
        print("MRR (raw): {:.6f}".format(mrr.item()))

        for hit in hits:
            avg_count = torch.mean((ranks <= hit).float())
            print("Hits (raw) @ {}: {:.6f}".format(hit, avg_count.item()))
    return mrr.item()

#######################################################################
#
# Utility functions for evaluations (filtered)
#
#######################################################################

def filter_o(triplets_to_filter, target_s, target_r, target_o, num_entities):
    target_s, target_r, target_o = int(target_s), int(target_r), int(target_o)
    filtered_o = []
    # Do not filter out the test triplet, since we want to predict on it
    if (target_s, target_r, target_o) in triplets_to_filter:
        triplets_to_filter.remove((target_s, target_r, target_o))
    # Do not consider an object if it is part of a triplet to filter
    for o in range(num_entities):
        if (target_s, target_r, o) not in triplets_to_filter:
            filtered_o.append(o)
    return torch.LongTensor(filtered_o)

def filter_s(triplets_to_filter, target_s, target_r, target_o, num_entities):
    target_s, target_r, target_o = int(target_s), int(target_r), int(target_o)
    filtered_s = []
    # Do not filter out the test triplet, since we want to predict on it
    if (target_s, target_r, target_o) in triplets_to_filter:
        triplets_to_filter.remove((target_s, target_r, target_o))
    # Do not consider a subject if it is part of a triplet to filter
    for s in range(num_entities):
        if (s, target_r, target_o) not in triplets_to_filter:
            filtered_s.append(s)
    return torch.LongTensor(filtered_s)

def perturb_o_and_get_filtered_rank(embedding, w, s, r, o, test_size, triplets_to_filter):
    """ Perturb object in the triplets
    """
    num_entities = embedding.shape[0]
    ranks = []
    for idx in range(test_size):
        if idx % 100 == 0:
            print("test triplet {} / {}".format(idx, test_size))
        target_s = s[idx]
        target_r = r[idx]
        target_o = o[idx]
        filtered_o = filter_o(triplets_to_filter, target_s, target_r, target_o, num_entities)
        target_o_idx = int((filtered_o == target_o).nonzero())
        emb_s = embedding[target_s]
        emb_r = w[target_r]
        emb_o = embedding[filtered_o]
        emb_triplet = emb_s * emb_r * emb_o
        scores = torch.sigmoid(torch.sum(emb_triplet, dim=1))
        _, indices = torch.sort(scores, descending=True)
        rank = int((indices == target_o_idx).nonzero())
        ranks.append(rank)
    return torch.LongTensor(ranks)

def perturb_s_and_get_filtered_rank(embedding, w, s, r, o, test_size, triplets_to_filter):
    """ Perturb subject in the triplets
    """
    num_entities = embedding.shape[0]
    ranks = []
    for idx in range(test_size):
        if idx % 100 == 0:
            print("test triplet {} / {}".format(idx, test_size))
        target_s = s[idx]
        target_r = r[idx]
        target_o = o[idx]
        filtered_s = filter_s(triplets_to_filter, target_s, target_r, target_o, num_entities)
        target_s_idx = int((filtered_s == target_s).nonzero())
        emb_s = embedding[filtered_s]
        emb_r = w[target_r]
        emb_o = embedding[target_o]
        emb_triplet = emb_s * emb_r * emb_o
        scores = torch.sigmoid(torch.sum(emb_triplet, dim=1))
        _, indices = torch.sort(scores, descending=True)
        rank = int((indices == target_s_idx).nonzero())
        ranks.append(rank)
    return torch.LongTensor(ranks)

def calc_filtered_mrr(embedding, w, train_triplets, valid_triplets, test_triplets, hits=[]):
    with torch.no_grad():
        s = test_triplets[:, 0]
        r = test_triplets[:, 1]
        o = test_triplets[:, 2]
        test_size = test_triplets.shape[0]

        triplets_to_filter = torch.cat([train_triplets, valid_triplets, test_triplets]).tolist()
        triplets_to_filter = {tuple(triplet) for triplet in triplets_to_filter}
        print('Perturbing subject...')
        ranks_s = perturb_s_and_get_filtered_rank(embedding, w, s, r, o, test_size, triplets_to_filter)
        print('Perturbing object...')
        ranks_o = perturb_o_and_get_filtered_rank(embedding, w, s, r, o, test_size, triplets_to_filter)

        ranks = torch.cat([ranks_s, ranks_o])
        ranks += 1 # change to 1-indexed

        mrr = torch.mean(1.0 / ranks.float())
        print("MRR (filtered): {:.6f}".format(mrr.item()))

        for hit in hits:
            avg_count = torch.mean((ranks <= hit).float())
            print("Hits (filtered) @ {}: {:.6f}".format(hit, avg_count.item()))
    return mrr.item()

#######################################################################
#
# Main evaluation function
#
#######################################################################

def calc_mrr(embedding, w, train_triplets, valid_triplets, test_triplets, hits=[], eval_bz=100, eval_p="filtered"):
    if eval_p == "filtered":
        mrr = calc_filtered_mrr(embedding, w, train_triplets, valid_triplets, test_triplets, hits)
    else:
        mrr = calc_raw_mrr(embedding, w, test_triplets, hits, eval_bz)
    return mrr

In [52]:
# build test graph
test_graph, test_rel, test_norm = build_test_graph(
    num_nodes, num_rels, train_data)
test_deg = test_graph.in_degrees(range(test_graph.number_of_nodes())).float().view(-1,1)
test_node_id = torch.arange(0, num_nodes, dtype=torch.long).view(-1, 1)
test_rel = torch.from_numpy(test_rel)
test_norm = node_norm_to_edge_norm(test_graph, torch.from_numpy(test_norm).view(-1, 1))

Test graph:


C:\Aplikasi\Anaconda\lib\site-packages\ipykernel_launcher.py:106: RuntimeWarning: divide by zero encountered in true_divide


# nodes: 14541, # edges: 544230


In [53]:
 # build adj list and calculate degrees for sampling
adj_list, degrees = get_adj_and_degrees(num_nodes, train_data)

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [1]:
import time
model_state_file = 'model_state.pth'
forward_time = []
backward_time = []
use_cuda=False
# training loop
print("start training...")

epoch = 0
best_mrr = 0
while True:
    model.train()
    epoch += 1

    # perform edge neighborhood sampling to generate training graph and data
    g, node_id, edge_type, node_norm, data, labels = \
        generate_sampled_graph_and_labels(
            train_data, 30000, 0.5,
            num_rels, adj_list, degrees, 10,
            'uniform')
    print("Done edge sampling")

        # set node/edge feature
    node_id = torch.from_numpy(node_id).view(-1, 1).long()
    edge_type = torch.from_numpy(edge_type)
    edge_norm = node_norm_to_edge_norm(g, torch.from_numpy(node_norm).view(-1, 1))
    data, labels = torch.from_numpy(data), torch.from_numpy(labels)
    deg = g.in_degrees(range(g.number_of_nodes())).float().view(-1, 1)
    if use_cuda:
        node_id, deg = node_id.cuda(), deg.cuda()
        edge_type, edge_norm = edge_type.cuda(), edge_norm.cuda()
        data, labels = data.cuda(), labels.cuda()

    t0 = time.time()
    embed = model(g, node_id, edge_type, edge_norm)
    loss = model.get_loss(g, embed, data, labels)
    t1 = time.time()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # clip gradients
    optimizer.step()
    t2 = time.time()

    forward_time.append(t1 - t0)
    backward_time.append(t2 - t1)
    print("Epoch {:04d} | Loss {:.4f} | Best MRR {:.4f} | Forward {:.4f}s | Backward {:.4f}s".
            format(epoch, loss.item(), best_mrr, forward_time[-1], backward_time[-1]))

    optimizer.zero_grad()

        # validation
    if epoch % 2 == 0:
        # perform validation on CPU because full graph is too large
        if use_cuda:
            model.cpu()
        model.eval()
        print("start eval")
        embed = model(test_graph, test_node_id, test_rel, test_norm)
        mrr = utils.calc_mrr(embed, model.w_relation, torch.LongTensor(train_data),
                                valid_data, test_data, hits=[1, 3, 10], eval_bz=500,
                                eval_p='filtered')
        # save best model
        if mrr < best_mrr:
            if epoch >= 10:
                break
        else:
            best_mrr = mrr
            torch.save({'state_dict': model.state_dict(), 'epoch': epoch},
                        model_state_file)
        if use_cuda:
            model.cuda()

print("training done")
print("Mean forward time: {:4f}s".format(np.mean(forward_time)))
print("Mean Backward time: {:4f}s".format(np.mean(backward_time)))

start training...


NameError: name 'model' is not defined

In [38]:
g

DGLGraph(num_nodes=11774, num_edges=30000,
         ndata_schemes={}
         edata_schemes={})

In [39]:
num_nodes

14541

In [44]:
g, len(node_id), len(edge_type), len(edge_norm)

(DGLGraph(num_nodes=11752, num_edges=30000,
          ndata_schemes={}
          edata_schemes={}), 11752, 30000, 30000)